## Задание


**Задание 1. Написать теггер на данных с русским языком**

- Проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
- Написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
- Сравнить все реализованные методы сделать выводы

**Задание 2. Проверить насколько хорошо работает NER**

данные брать из http://www.labinform.ru/pub/named_entities/

- Проверить NER из nltk/spacy/deeppavlov
- Написать свой нер попробовать разные подходы
    - Передаём в сетку токен и его соседей
    - Передаём в сетку только токен
    - Свой вариант
- Сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)

---

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import pymorphy2
import re

from tqdm.notebook import tqdm

from corus import load_ne5

import matplotlib
%matplotlib inline

In [ ]:
df = pd.read_csv('negative.csv', sep = ';', usecols = [3], names = ['text'])
df.head(5)

In [ ]:
words_regex = re.compile('\w+')

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')

morph = pymorphy2.MorphAnalyzer()

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')


def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [ ]:
preprocessed_text = list(tqdm(map(preprocess, df['text']), total = len(df)))

In [ ]:
words = list(tqdm(map(find_words, df['text']), total = len(df)))

In [ ]:
train_data = words[1:int(len(words) * 0.9)]
test_data = words[int(len(words) * 0.9):]
test_sent = words[0]

In [ ]:
train_data = [nltk.pos_tag(i, lang = 'rus') for i in train_data]
test_data = [nltk.pos_tag(i, lang = 'rus') for i in test_data]

In [ ]:
unigram_tagger = UnigramTagger(train_data)
unigram_tagger.evaluate(test_data)

In [ ]:
bigram_tagger = BigramTagger(train_data, backoff = unigram_tagger)
bigram_tagger.evaluate(test_data)

In [ ]:
trigram_tagger = TrigramTagger(train_data, backoff = bigram_tagger)
trigram_tagger.evaluate(test_data)

In [ ]:
def backoff_tagger(train_sents, tagger_classes, backoff = None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff = backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data)

---

In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

In [ ]:
!unzip collection5.zip

In [ ]:
text = "Россия рассчитывает на конструктивное воздействие США на Грузию 04/08/2008 12:08 МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом. С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений, - говорится в сообщении. Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона, - сообщил МИД России" 

In [ ]:
import re

In [ ]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
# spacy

import spacy
from spacy import displacy

nlp = spacy.load('ru_core_news_sm')

docs = nlp(text)
displacy.render(docs, jupyter=True, style='ent')